In [1]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics_class import Metrics
import shutil
from predict_maskpls import RefinerModel

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/cedric/anaconda3/envs/torch/lib/python3.9/site-packages/MinkowskiEngine-0.5.4-py3.9-linux-x86_64.egg/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


Here we define the dataset depending on kitti sequence!

In [2]:
DATASET_PATH = os.path.join('/media/cedric/Datasets1/semantic_kitti/')
SEQUENCE_NUM = 7

ind_start = 0
ind_end = 1100
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False 
out_folder_ncuts = 'test_data/'
if os.path.exists(out_folder_ncuts):
        shutil.rmtree(out_folder_ncuts)
os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [3]:

'''
if os.path.exists('{out_folder}all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)
'''

"\nif os.path.exists('{out_folder}all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:\n        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,\n                                major_voxel_size=major_voxel_size,icp=False,\n                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,\n                                ground_segmentation_method=ground_segmentation_method)\n"

This cell can be ignored after first run as outputs are stored 

In [4]:
##load data if already stored 

if os.path.exists(f'{out_folder}pcd_ground_minor.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)
        
        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}kitti_labels_preprocessed.npz',panoptic_nonground=kitti_labels['panoptic_nonground'],
                                                panoptic_ground=kitti_labels['panoptic_ground'],
                                                instance_nonground=kitti_labels['instance_nonground'],
                                                instance_ground=kitti_labels['instance_ground'],
                                                seg_ground = kitti_labels['seg_ground'],
                                                seg_nonground=kitti_labels['seg_nonground']
                                                )


In [5]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}.pcd')

kitti_labels_orig = {}
with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}.npz') as data :
        kitti_labels_orig['panoptic_ground'] = data['panoptic_ground']
        kitti_labels_orig['panoptic_nonground'] = data['panoptic_nonground']
        kitti_labels_orig['instance_ground'] = data['instance_ground']
        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
        kitti_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_{0}.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

In [6]:
'''
pcd_new = o3d.geometry.PointCloud()
pts_num = 1000000
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])

map_labelled = color_pcd_by_labels(pcd_new,\
                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)
'''

"\npcd_new = o3d.geometry.PointCloud()\npts_num = 1000000\npcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])\n\nmap_labelled = color_pcd_by_labels(pcd_new,                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))\n\no3d.visualization.draw_geometries([map_labelled])\n#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)\n"

Now we subsample the poses based on a voxel_size

In [7]:
poses, positions, \
sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)

Now we can split the point cloud into chunks based on a tbd chunk_size

In [8]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)

Downsampled from (523285, 3) to (7646, 3) points (non-ground)
Downsampled from (257594, 3) to (5502, 3) points (ground)
Downsampled from (392323, 3) to (6659, 3) points (non-ground)
Downsampled from (273555, 3) to (4539, 3) points (ground)
Downsampled from (429859, 3) to (7470, 3) points (non-ground)
Downsampled from (291612, 3) to (6157, 3) points (ground)
Downsampled from (381642, 3) to (5813, 3) points (non-ground)
Downsampled from (255212, 3) to (4655, 3) points (ground)
Downsampled from (414475, 3) to (7677, 3) points (non-ground)
Downsampled from (237620, 3) to (4305, 3) points (ground)
Downsampled from (463157, 3) to (8880, 3) points (non-ground)
Downsampled from (226253, 3) to (5086, 3) points (ground)
Downsampled from (506123, 3) to (9688, 3) points (non-ground)
Downsampled from (228215, 3) to (4918, 3) points (ground)
Downsampled from (389712, 3) to (6633, 3) points (non-ground)
Downsampled from (308744, 3) to (4290, 3) points (ground)
Downsampled from (280190, 3) to (4670, 3

In [9]:
import sklearn
from sklearn.cluster import DBSCAN

def DBSCAN_clustering_logic(cur_pcd, pcd_all, eps=1.0, min_samples=100):
    """
    Perform DBSCAN clustering on the point cloud data.

    :param cur_pcd: Current point cloud for clustering.
    :param pcd_all: All point cloud data.
    :param eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
    :param min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
    :return: Cluster labels for each point in the point cloud.
    """
    not_road_points = np.asarray(cur_pcd.points)
    #clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(not_road_points)
    clustering = DBSCAN(min_cluster_size=100).fit(not_road_points)
    labels_not_road = clustering.labels_
    colors_gen = generate_random_colors(500)
    
    # Reproject cluster labels to the original point cloud size
    cluster_labels = np.ones((len(pcd_all.points), 1)) * -1
    labels_non_ground = kDTree_1NN_feature_reprojection(cluster_labels, pcd_all, labels_not_road.reshape(-1,1), cur_pcd)
    colors = np.zeros((labels_non_ground.shape[0],3))
    unique_labels = list(np.unique(labels_non_ground))
    for j in unique_labels:
            cur_idcs = np.where(labels_non_ground == j)[0]
            
            colors[cur_idcs] = np.array(colors_gen[unique_labels.index(j)])
    pcd_all.colors = o3d.utility.Vector3dVector(colors / 255.)
    return pcd_all

def dbscan_clustering(pcd_chunks_major_downsampled, pcds, center_ids,ground_clouds):
    labels_clustering = []
    for i in range(len(pcd_chunks_major_downsampled)):
        cur_pcd = pcd_chunks_major_downsampled[i]
        pcd_all = pcds[i]
        ground_cloud = ground_clouds[i]
        cluster_labels = DBSCAN_clustering_logic(cur_pcd, pcd_all)  # Implement your DBSCAN logic here
        ground_labels = np.ones((np.asarray(ground_cloud.points).shape[0],1)) * -1
        cluster_labels = np.concatenate((cluster_labels,ground_labels),0)
        labels_clustering.append(cluster_labels)
        
    return labels_clustering


In [10]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])


    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
        #if labels[i] != (-1):
        #    pcd_colored.colors[i] = np.array(colors[labels[i]]) / 255
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/ 255)
    return pcd_colored

In [11]:
alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.0
proximity_threshold = 1.0



out_dbscan = 'out_dbscan/'
if os.path.exists(out_dbscan) == True : 
        shutil.rmtree(out_dbscan)
        
os.makedirs(out_dbscan)

out_kitti = 'out_kitti/'
if os.path.exists(out_kitti) == True : 
        shutil.rmtree(out_kitti)

os.makedirs(out_kitti)
        
out_kitti_instance = 'out_kitti_instance/'
#if os.path.exists(out_kitti_instance) == True : 
#        shutil.rmtree(out_kitti_instance)
#os.makedirs(out_kitti_instance)

        
out_refined = 'out_refined/'
if os.path.exists(out_refined) == True : 
        shutil.rmtree(out_refined)
os.makedirs(out_refined)

limit = 2 ##use this for experiments to run limit chunks numberss

maskpls = RefinerModel()
eval_refiner = True 

patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
for sequence in range(len(center_ids)):
        if NCUT_ground == False : 
                
                name = str(center_ids[sequence]).zfill(6) + '.pcd'
                if eval_refiner == False : 
                        merged_chunk,file_name, pcd_chunk, pcd_chunk_ground, inliers_ground = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                                pcd_nonground_chunks_major_downsampling,
                                pcd_nonground_minor,T_pcd,center_positions,center_ids,
                                positions,first_position,sampled_indices_global,
                                chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                                alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                                proximity_threshold=proximity_threshold,
                                out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                                patchwise_indices=patchwise_indices)
                        o3d.io.write_point_cloud(file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                        kitti_labels['ground']['panoptic'][sequence] = kitti_labels['ground']['panoptic'][sequence][inliers_ground]
                        kitti_labels['ground']['instance'][sequence] = kitti_labels['ground']['instance'][sequence][inliers_ground]
                        #kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['panoptic'][sequence].reshape(-1,),
                        #                        colors=colors,gt_labels=kitti_labels_orig['panoptic_nonground'])
                        #kitti_chunk_instance = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['instance'][sequence].reshape(-1,),
                        #                        colors=colors,gt_labels=kitti_labels_orig['instance_nonground'])
                        #o3d.io.write_point_cloud(out_kitti + name, kitti_chunk + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                        #o3d.io.write_point_cloud(out_kitti_instance + name, kitti_chunk_instance + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                else :  
                        print("sequence",sequence)
                        cur_pcd = o3d.io.read_point_cloud(out_kitti_instance + name)
                        predicted_refined = maskpls.forward_and_project(cur_pcd)
                        #maskpls.project_pseudo_labels(cur_pcd,center_ids[sequence],name)
                        o3d.io.write_point_cloud(out_refined + name, predicted_refined,write_ascii=False, compressed=False, print_progress=False)
                
                #pcd_dbscan = DBSCAN_clustering_logic(pcd_nonground_chunks_major_downsampling[sequence],
                #                                pcd_nonground_chunks[sequence],
                #                                eps=0.6, min_samples=10)
                
                
                #kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,kitti_labels['ground']['instance'][sequence].reshape(-1,))
                #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                
                
                
        else : 
                obstacle_out,file_name = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=True,sequence=sequence,
                        patchwise_indices=patchwise_indices)
                
                
                ground_out,file_name = ncuts_chunk(dataset,indices_ground,pcd_ground_chunks,None,
                        pcd_ground_chunks_major_downsampling,
                        pcd_ground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=True,sequence=sequence,
                        patchwise_indices=patchwise_indices)

                o3d.io.write_point_cloud(file_name, obstacle_out + ground_out, write_ascii=False, compressed=False, print_progress=False)

                print("Pointcloud written to file")

        
        
        

sequence 0
sequence 1
sequence 2
sequence 3
sequence 4
sequence 5
sequence 6
sequence 7
sequence 8
sequence 9
sequence 10
sequence 11
sequence 12
sequence 13
sequence 14
sequence 15
sequence 16
sequence 17
sequence 18
sequence 19
sequence 20
sequence 21
sequence 22
sequence 23
sequence 24
sequence 25
sequence 26
sequence 27
sequence 28
sequence 29


In [12]:
def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds




def merge_unite_gt(chunks):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:
        merge += new_chunk
    
    merge.remove_duplicated_points()
    return merge 

def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]


def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels

In [13]:
out_dbscan = 'out_dbscan/'
out_kitti = 'out_kitti/'
out_kitti_instance = 'out_kitti_instance/'


point_clouds = get_merge_pcds(out_folder_ncuts)
#point_clouds_dbscan = get_merge_pcds(out_dbscan)
#point_clouds_kitti = get_merge_pcds(out_kitti)[:-1]
point_clouds_kitti_instances = get_merge_pcds(out_kitti_instance)[:-1]
point_clouds_refined = get_merge_pcds(out_refined)[:-1]

#merge = merge_chunks_unite_instances(point_clouds)
#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)
#merge_kitti = merge_unite_gt(point_clouds_kitti)
merge_kitti_instance = merge_unite_gt(point_clouds_kitti_instances)
merge_refined = merge_chunks_unite_instances(point_clouds_refined)

#o3d.io.write_point_cloud(out_folder + "merge_part.pcd", merge, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + "merge_refined.pcd", merge_refined, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_dbscan.pcd", merge_dbscan, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti.pcd", merge_kitti, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance.pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)

#unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
unique_colors, labels_refined = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)




[]
['000062.pcd', '000090.pcd', '000121.pcd', '000161.pcd', '000190.pcd', '000216.pcd', '000245.pcd', '000275.pcd', '000330.pcd', '000366.pcd', '000389.pcd', '000412.pcd', '000437.pcd', '000477.pcd', '000513.pcd', '000537.pcd', '000562.pcd', '000584.pcd', '000606.pcd', '000634.pcd', '000769.pcd', '000790.pcd', '000809.pcd', '000829.pcd', '000850.pcd', '000872.pcd', '000906.pcd', '000944.pcd', '000975.pcd', '001029.pcd']
['000062.pcd', '000090.pcd', '000121.pcd', '000161.pcd', '000190.pcd', '000216.pcd', '000245.pcd', '000275.pcd', '000330.pcd', '000366.pcd', '000389.pcd', '000412.pcd', '000437.pcd', '000477.pcd', '000513.pcd', '000537.pcd', '000562.pcd', '000584.pcd', '000606.pcd', '000634.pcd', '000769.pcd', '000790.pcd', '000809.pcd', '000829.pcd', '000850.pcd', '000872.pcd', '000906.pcd', '000944.pcd', '000975.pcd', '001029.pcd']


In [14]:
'''
optional cell for loading stored files
import open3d as o3d

out_folder = 'pcd_preprocessed/'
merge = o3d.io.read_point_cloud(out_folder + 'merge_part.pcd')
merge_dbscan = o3d.io.read_point_cloud(out_folder + 'merge_part_dbscan.pcd')
merge_kitti = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti.pcd')
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')
merge_exp2 = o3d.io.read_point_cloud(out_folder + 'merge_exp2.pcd')

print(merge)
print(merge_kitti)
print(merge_exp2)
'''

"\noptional cell for loading stored files\nimport open3d as o3d\n\nout_folder = 'pcd_preprocessed/'\nmerge = o3d.io.read_point_cloud(out_folder + 'merge_part.pcd')\nmerge_dbscan = o3d.io.read_point_cloud(out_folder + 'merge_part_dbscan.pcd')\nmerge_kitti = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti.pcd')\nmerge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')\nmerge_exp2 = o3d.io.read_point_cloud(out_folder + 'merge_exp2.pcd')\n\nprint(merge)\nprint(merge_kitti)\nprint(merge_exp2)\n"

In [15]:
#new_ncuts_labels = remove_semantics(labels_kitti,labels_ncuts)
new_refined = remove_semantics(labels_kitti,labels_refined)


metrics_ncuts = Metrics(name='ncuts')
metrics_refined = Metrics(name='refined')
#metrics_dbscan = Metrics(name='dbscan')
metrics_test = Metrics(name='test')

#metrics_ncuts.update_stats(new_ncuts_labels,labels_kitti)
metrics_refined.update_stats(new_refined,labels_kitti)

#metrics_dbscan.update_stats(new_dbscan_labels,labels_kitti)
#metrics_dbscan.compute_all_aps()
#metrics_ncuts.compute_all_aps()
metrics_refined.compute_all_aps()

#merge_vis = color_pcd_by_labels(merge,new_ncuts_labels)
#o3d.visualization.draw_geometries([merge_vis])


{'panoptic': 0.7887763627032877, 'precision': 0.8775510204081632, 'recall': 0.8037383177570093, 'fScore': 0.8390243902439024, 'usr': 0.04081632653061224, 'osr': 0.009345794392523364, 'noise': 0.10204081632653061, 'missed': 0.14953271028037382, 'mean': 0.9401113625242673}
APs for  refined
mAP@0.25  0.8598130841121495
mAP@0.5  0.8037383177570093
mAP  0.7487019730010385


In [16]:
out_folder = 'pcd_preprocessed/'
merge_refined = o3d.io.read_point_cloud(out_folder + 'merge_refined.pcd')
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')

unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
unique_colors, labels_refined = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)

new_refined = remove_semantics(labels_kitti,labels_refined)
o3d.visualization.draw_geometries([color_pcd_by_labels(merge_refined,new_refined)])

In [17]:
o3d.visualization.draw_geometries([color_pcd_by_labels(merge_refined,new_refined)])